In [1]:
from pyspark.sql import SparkSession
import json

In [2]:
cassandra_host = "192.168.31.254"
cassandra_user = "cassandra"
cassandra_pwd  = "cassandra"
cassandra_port = 9042
key_space      = "LogAnalysis"
table_name     = "ApacheLogs"
kafka_server   = "192.168.31.254:9092"
kafka_topic    = "access"

In [3]:
#Spark Session creation configured to interact with MongoDB
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,com.datastax.spark:spark-cassandra-connector_2.12:3.0.0,com.datastax.spark:spark-cassandra-connector-driver_2.12:3.0.0").\
config("spark.cassandra.connection.host",cassandra_host).\
config("spark.cassandra.auth.username",cassandra_user).\
config("spark.cassandra.auth.password",cassandra_pwd).\
getOrCreate()

:: loading settings :: url = jar:file:/Users/macpro2014/miniconda3/envs/data-engineering/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/macpro2014/.ivy2/cache
The jars for the packages stored in: /Users/macpro2014/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector-driver_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0c4b8917-1103-427d-bb5c-c8da05af6759;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found com.datastax.spark#spark

In [4]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, TimestampType, IntegerType, LongType

# Read Kafka messages
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "latest") \
    .load()

In [5]:
df = df.selectExpr("CAST(value AS STRING)")  
df.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([StructField("message", StringType())])
df = df.withColumn("value", from_json("value", schema))
df = df.select("value.message")

In [ ]:
# print out the log file
df.writeStream.format("console").outputMode("append").start().awaitTermination()